In [19]:
import pickle
import torch
import cv2
from torch.utils.data import Dataset, DataLoader
import numpy as np
#https://pytorch.org/hub/ultralytics_yolov5/

# with open("CUDatamap_line2.pkl","wb") as f:
#     pickle.dump(data_line_map,f) #Store the datamap for further processig
# with open("CUDatamap_coef.pkl","wb") as f:
#     pickle.dump(data_coef_map,f) #Store the datamap for further processig

with open("CUDatamap_line2.pkl","rb") as f:
    data_line_map = pickle.load(f)
with open("CUDatamap_stline2.pkl","rb") as f:
    data_stline_map = pickle.load(f)

with open("CUDatamap_coef.pkl","rb") as f:
    data_coef_map = pickle.load(f)



y_col = np.linspace(400,580,19)

class LaneDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, data_line_map,data_stline_map,scale=None):
        
        self.new_data_keys = list(data_line_map.keys())
        self.data_line_map = data_line_map
        self.data_stline_map = data_stline_map
        self.scale = scale
        image_loc = self.new_data_keys[0]
        image = cv2.imread(image_loc)
        height,width,channel = image.shape
        self.height = height
        self.width = width
        

  

    def __len__(self):
        return len(self.new_data_keys)
    
    


    def get_lane_angle(self,myline):
        global y_col #its actually row
        row = y_col[-1]
        col = myline[-1]
        
        new_col = col - self.width//2
        new_row = row + self.height

        angle = -np.arctan2(new_col,new_row)
        # if angle<0:
        #     angle = 2*np.pi+angle
        
        return angle

    def get_lane_bins(self,stline):
        angle = self.get_lane_angle(stline)
        angle = np.clip(angle,-1,1) 
        mybin = int(round(angle/0.125))+8 #17 bins -1 to 1 as 0 to 17 

        return mybin,angle    

        
    def get_regression_confidence_mat(self,stlines,lines):
        regression_mat = np.zeros((19,17),dtype=np.float32) #19 rows and 17 for 17 bins
        confidence_mat = np.zeros((17,),dtype=np.float32)
        for stline,line in zip(stlines,lines):
            mybin,angle = self.get_lane_bins(stline=stline)
            regression_mat[:,mybin] = line
            confidence_mat[mybin] = 1
        return regression_mat.flatten(),confidence_mat

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        image_loc = self.new_data_keys[idx]
        lines = self.data_line_map[image_loc] #Since
        image = cv2.imread(image_loc)
        stlines = self.data_stline_map[image_loc]

        
        #visualize_bundle = (image,lines,stlines,idx) #use only for visualization
        

        image_tensor = torch.tensor(image,dtype=torch.float32)/255 #convert to tensor and normalise
        image_tensor2 = image_tensor.permute(2,0,1) #bring the channel to front

        regression_mat,confidence_mat  = self.get_regression_confidence_mat(stlines,lines)
        regression_mat_tensor = torch.tensor(regression_mat,dtype=torch.float32)
        confidence_mat_tensor = torch.tensor(confidence_mat,dtype=torch.float32)

        #print(image_loc,sum(regression_mat))
        

        return image_tensor2,regression_mat_tensor,confidence_mat_tensor

In [15]:
import torch.nn as nn
from torchvision import models

In [3]:
# Load the VGG16 model from torchvision
vgg16 = models.vgg16(pretrained=True)

/home/akshayd/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/akshayd/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /home/akshayd/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [10:15<00:00, 899kB/s]    


In [4]:
torch.save(vgg16,"vgg16.pt")

In [5]:
hash(vgg16)

8756090160951

In [6]:
vgg16 = torch.load("vgg16.pt")

In [7]:
hash(vgg16)

8756090053527

In [35]:
conv_net_features = nn.Sequential(
            vgg16.features[0],  # Conv1_1
            vgg16.features[1],  # ReLU1_1
            vgg16.features[2],  # Conv1_2
            vgg16.features[3],  # ReLU1_2
            vgg16.features[4],  # 
            vgg16.features[5],  # 
            vgg16.features[6],  # 
            vgg16.features[7],  # 
            vgg16.features[8],  # 
            vgg16.features[9],  # 
        )

In [36]:
lane_dataset = LaneDataset(data_line_map=data_line_map,data_stline_map=data_stline_map)
dataloader = DataLoader(lane_dataset, batch_size=1,shuffle=True, num_workers=4)

In [37]:
count=0
for batch in dataloader:
    image_tensor,regression_mat_tensor,confidence_mat_tensor = batch
    output = conv_net_features(image_tensor)
    count+=1
    print("-----------------\n-------------------\n")
    if count==1:
        break

-----------------
-------------------



In [38]:
output.shape

torch.Size([1, 128, 147, 410])

In [26]:
regression_mat_tensor.shape

torch.Size([1, 323])

In [27]:
confidence_mat_tensor.shape

torch.Size([1, 17])

In [34]:
vgg16.features

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (17): Conv2d(256, 512, kernel_si